In [1]:
import numpy as np


In [2]:
from pathlib import Path

WORKDIR = Path.cwd().absolute()
INPUTFILE = WORKDIR / "day-14.txt"

with INPUTFILE.open("r") as file:
    inputs = [line.strip() for line in file.readlines()]

_inputs = [
    "498,4 -> 498,6 -> 496,6",
    "503,4 -> 502,4 -> 502,9 -> 494,9"
]

inputs[0:8]

['521,154 -> 526,154',
 '474,44 -> 474,38 -> 474,44 -> 476,44 -> 476,39 -> 476,44 -> 478,44 -> 478,34 -> 478,44 -> 480,44 -> 480,40 -> 480,44 -> 482,44 -> 482,36 -> 482,44 -> 484,44 -> 484,34 -> 484,44 -> 486,44 -> 486,38 -> 486,44 -> 488,44 -> 488,36 -> 488,44 -> 490,44 -> 490,36 -> 490,44',
 '499,13 -> 499,17 -> 493,17 -> 493,24 -> 507,24 -> 507,17 -> 501,17 -> 501,13',
 '497,80 -> 497,83 -> 489,83 -> 489,87 -> 505,87 -> 505,83 -> 501,83 -> 501,80',
 '471,77 -> 475,77',
 '502,122 -> 502,115 -> 502,122 -> 504,122 -> 504,114 -> 504,122 -> 506,122 -> 506,115 -> 506,122 -> 508,122 -> 508,119 -> 508,122 -> 510,122 -> 510,113 -> 510,122 -> 512,122 -> 512,116 -> 512,122 -> 514,122 -> 514,115 -> 514,122 -> 516,122 -> 516,113 -> 516,122 -> 518,122 -> 518,117 -> 518,122',
 '545,157 -> 545,160 -> 544,160 -> 544,167 -> 556,167 -> 556,160 -> 549,160 -> 549,157',
 '477,73 -> 481,73']

In [3]:
xs = set()
ys = set()
for line in inputs:
    line = line.split(" -> ")
    for pair in line:
        x, y = list(map(int, pair.split(",")))
        xs.add(x)
        ys.add(y)

print(f"Minimum x = {min(xs)}, y = {min(ys)}")
print(f"Maximum x = {max(xs)}, y = {max(ys)}")

x_offset = min(xs) - 1
width = max(xs) - min(xs) + 1 + 2   # +1 because inclusive, +2 for margin/buffer
height = max(ys) + 1
print(f"{x_offset=}, {width=}, {height=}")


Minimum x = 464, y = 13
Maximum x = 556, y = 167
x_offset=463, width=95, height=168


In [8]:
AIR = ord(" ")
cavern = np.full((height, width), AIR, dtype=np.uint8)
ROCK = ord("#")
for line in inputs:
    line = line.split(" -> ")
    for i, pair in enumerate(line):
        if i == 0:
            x0, y0 = list(map(int, pair.split(",")))
            x0 -= x_offset
        else:
            x1, y1 = list(map(int, pair.split(",")))
            x1 -= x_offset
            if x1 == x0:
                for y in range(min(y0, y1), max(y0,y1)+1):
                    cavern[y, x0] = ROCK
            elif y1 == y0:
                for x in range(min(x0, x1), max(x0, x1)+1):
                    cavern[y0, x] = ROCK
            else:
                raise RuntimeError(f"({x0},{y0}) -> ({x1},{y1}) is neither horizontal nor vertical")
            x0, y0 = x1, y1

def print_cavern(filepath: Path) -> None:
    with filepath.open("w") as file:
        file.writelines(("".join(map(chr, row))+"\n" for row in cavern))

print_cavern(WORKDIR / "day-14-cavern1a.txt")


In [9]:
from collections import namedtuple

Cell = namedtuple("Cell", ["x", "y"])
SAND = ord("·")

grains = 0
path = [Cell(500-x_offset, 0)]
while True:
    position = path.pop()
    grains += 1
    while position.y < (cavern.shape[0] - 1):
        if cavern[position.y+1, position.x] == AIR:
            path.append(position)
            position = Cell(position.x, position.y+1)
        elif cavern[position.y+1, position.x-1] == AIR:
            path.append(position)
            position = Cell(position.x-1, position.y+1)
        elif cavern[position.y+1, position.x+1] == AIR:
            path.append(position)
            position = Cell(position.x+1, position.y+1)
        else:
            # final resting spot
            cavern[position.y, position.x] = SAND
            break
    if position.y == (cavern.shape[0] - 1):
        break

print(f"Part 1: {grains-1} grains of sand came to rest")

print_cavern(WORKDIR / "day-14-cavern1b.txt")


Part 1: 994 grains of sand came to rest


In [10]:
x_offset = 0
width = 1024
height += 2
print(f"{x_offset=}, {width=}, {height=}")

cavern = np.full((height, width), AIR, dtype=np.uint8)

for line in inputs:
    line = line.split(" -> ")
    for i, pair in enumerate(line):
        if i == 0:
            x0, y0 = list(map(int, pair.split(",")))
            x0 -= x_offset
        else:
            x1, y1 = list(map(int, pair.split(",")))
            x1 -= x_offset
            if x1 == x0:
                for y in range(min(y0, y1), max(y0,y1)+1):
                    cavern[y, x0] = ROCK
            elif y1 == y0:
                for x in range(min(x0, x1), max(x0, x1)+1):
                    cavern[y0, x] = ROCK
            else:
                raise RuntimeError(f"({x0},{y0}) -> ({x1},{y1}) is neither horizontal nor vertical")
            x0, y0 = x1, y1

cavern[max(ys)+2,:] = ROCK

print_cavern(WORKDIR / "day-14-cavern2a.txt")


x_offset=0, width=1024, height=172


In [11]:
grains = 0
source = Cell(500, 0)
path = [source]
while True:
    position = path.pop()
    grains += 1
    # if grains % 100 == 0:
    #     print(f"{grains}...")
    if grains >= 60_000:
        break
    while position.y < (cavern.shape[0] - 1):
        if cavern[position.y+1, position.x] == AIR:
            path.append(position)
            position = Cell(position.x, position.y+1)
        elif cavern[position.y+1, position.x-1] == AIR:
            path.append(position)
            position = Cell(position.x-1, position.y+1)
        elif cavern[position.y+1, position.x+1] == AIR:
            path.append(position)
            position = Cell(position.x+1, position.y+1)
        else:
            # final resting spot
            cavern[position.y, position.x] = SAND
            break
    if position == source:
        break

print(f"Part 2: {grains} grains of sand came in")

print_cavern(WORKDIR / "day-14-cavern2b.txt")


Part 2: 26283 grains of sand came in
